# N-Gram Language Modeling Lab

## Introduction

In this lab, you will build an n-gram language model using a corpus from Project Gutenberg. You'll start by loading and tokenizing the corpus using a pre-existing tokenizer, constructing an n-gram frequency dictionary, and finally implementing beam search to generate text based on your model. Some parts of the code will be left for you to complete.

## 1. Load the Corpus

First, we'll load a text corpus from Project Gutenberg. For this lab, we'll use the text of "Alice's Adventures in Wonderland" by Lewis Carroll.

In [1]:
import urllib.request
import os

def download_corpus(url, filename):
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

In [2]:
corpus_url = "https://gutenberg.org/ebooks/11.txt.utf-8"
corpus_filename = "alice_in_wonderland.txt"

download_corpus(corpus_url, corpus_filename)

In [3]:
corpus = open(corpus_filename, 'r').read()

In [ ]:
corpus

In [ ]:
# Strip out unicode (for this demo)
corpus = corpus.encode('ascii', 'ignore').decode('utf-8')
corpus[:100]

In [ ]:
from textwrap import fill
print(fill(corpus[:100], drop_whitespace=False))

## 2. Tokenize the Corpus

In this section, you'll build a simple Byte Pair Encoding (BPE) tokenizer to split the text into tokens. BPE is a subword segmentation algorithm that iteratively merges the most frequent pair of bytes or characters.


In [ ]:
tokens = list(corpus)
print(tokens[:10])

In [ ]:
from collections import defaultdict

def get_stats(tokens):
    """Compute frequencies of adjacent pairs."""
    pairs = defaultdict(int)
    for i in range(len(tokens)-1):
        pairs[(tokens[i], tokens[i+1])] +=1
    return pairs

stats = get_stats(tokens)
stats

In [ ]:
n = 10
top_n_tokens = sorted(stats.items(), key=lambda x: x[1], reverse=True)[:n]
top_n_tokens

In [ ]:
most_frequent_token = max(stats, key=stats.get)
most_frequent_token

In [ ]:
def merge_vocab(pair, tokens):
    merged = []
    i = 0
    while i < len(tokens)-1:
        if (tokens[i], tokens[i+1]) == pair:
            merged.append(pair)
            i += 2
        else:
            merged.append(tokens[i])
            i += 1
    return merged

tokens = merge_vocab(most_frequent_token, tokens)
tokens[:10]

In [ ]:
def flatten_token(token):
    if isinstance(token, tuple):
        return ''.join(flatten_token(t) for t in token)
    else:
        return token

flatten_token((((' ', 't'), 'h'), 'e'))

In [ ]:
import ipywidgets
import tqdm.notebook as tq

def build_bpe_vocab(corpus, num_merges, verbose=False):
    """Build BPE vocabulary."""
    # Initialize tokens as characters with end-of-word token
    tokens = list(corpus)
    vocab_size = len(set(tokens))
    
    for i in tq.trange(num_merges, leave=False, desc='Building BPE vocab'):
        pairs = get_stats(tokens)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        tokens = merge_vocab(best, tokens)
        if verbose:
            tq.tqdm.write(f'Merge {i+1:<5}: {repr(flatten_token(best)):<10}')
    
    vocab = sorted([flatten_token(t) for t  in set(tokens)], key=len, reverse=True)
    return vocab

vocab = build_bpe_vocab(corpus, num_merges=10, verbose=True)
print("\nBPE Vocabulary (first 10 tokens):", vocab[:10])

In [ ]:
#vocab = build_bpe_vocab(corpus, num_merges=len(corpus) - 1000)
vocab = build_bpe_vocab(corpus, num_merges=1000) # (I am not patient, this may take a while)

In [ ]:
vocab[:10]

In [ ]:
print("Lengh of final vocab:", len(vocab))

In [ ]:
#TODO: Consider using a Trie data structure for efficient tokenization (e.g. marisa_trie)
def tokenize(input_str, vocab):
    tokens = []
    i = 0

    while i < len(input_str):
        # Try to match the longest token from the current position
        for token in vocab:
            if input_str[i:].startswith(token):
                tokens.append(token)
                i += len(token)
                break
        else:
            tokens.append(token)
            i += 1

    return tokens

bpe_tokens = tokenize(corpus, vocab) # Takes about 2m 30 seconds
print("First 10 tokens:", bpe_tokens[:10])

In [ ]:
print(corpus[:30])

In [47]:
# %pip install marisa-trie

In [ ]:
import marisa_trie

def tokenize(input_str, vocab):
    # Build the Trie from the vocabulary
    trie = marisa_trie.Trie(vocab)
    
    tokens = []
    i = 0

    while i < len(input_str):
        # Find all prefixes that match the current position
        prefixes = trie.prefixes(input_str[i:])
        
        if prefixes:
            # Choose the longest matching prefix
            longest_prefix = max(prefixes, key=len)
            tokens.append(longest_prefix)
            i += len(longest_prefix)  # Move forward by the length of the matched token
        else:
            tokens.append(input_str[i])
            i += 1  # No match, move to the next character

    return tokens


bpe_tokens = tokenize(corpus, vocab) # Takes about 2.5 seconds
print("First 10 tokens:", bpe_tokens[:10])  


## 3. Create N-Gram Frequency Dictionary

We'll create a hash table where each prefix (n-1 grams) maps to a dictionary of possible next tokens and their frequencies.

In [123]:
from collections import defaultdict, Counter

def build_ngram_freq(tokens, n=2):
    ngram_freq = defaultdict(Counter)
    for k in range(1, n+1):
        for i in range(len(tokens) - k):
            ngram = tokens[i:][:k]
            prefix = ngram[:-1]  
            next_token = ngram[-1]
            ngram_freq[tuple(prefix)][next_token] += 1
    return ngram_freq

In [ ]:
ngram_freq = build_ngram_freq(bpe_tokens, n=2)
print(f"Number of unique prefixes: {len(ngram_freq)}")
print("Sample prefix and possible next tokens with frequencies:")
sample_prefix = list(ngram_freq.keys())[0]
print(f"Prefix: {sample_prefix}")
print(ngram_freq[sample_prefix].most_common(5))

**Expected Output:**


```txt
    Number of unique prefixes: 1044
    Sample prefix and possible next tokens with frequencies:
    Prefix: ('The ',)
    [('Queen', 4), ('Mouse ', 4), ('Queen ', 4), ('P', 3), ('Rabbit ', 3)]
```


## 4. Beam Search Sampling

We'll implement beam search to generate text based on the n-gram model. Beam search keeps track of the top `k` sequences at each step to explore multiple hypotheses.

In [128]:
import math
import random

def beam_search_sampling(ngram_freq, n=2, beam_width=3, max_length=20, seed_prefix=None):
    """
    Generate text using beam search.
    
    Args:
        ngram_freq (dict): The n-gram frequency dictionary.
        beam_width (int): Number of sequences to keep at each step.
        max_length (int): Maximum number of tokens to generate.
        seed_prefix (list): Initial list of tokens to start the generation.
        
    Returns:
        list: The generated sequence of tokens.
    """
    if seed_prefix is None:
        # Start with a random prefix
        seed_prefix = []

    if not isinstance(seed_prefix, list):
        seed_prefix = [seed_prefix]

    sequences = [(seed_prefix, 0.0)]  # (sequence, score)
    
    for _ in range(max_length):
        all_candidates = []
        for seq, score in sequences:
            prefix = seq[-n+1:]  # For an n-gram model, the prefix is the last n-1 tokens
            next_tokens = ngram_freq.get(tuple(prefix), {})
            if not next_tokens:
                continue
            for token, freq in next_tokens.items():
                prob = freq / sum(next_tokens.values())
                candidate_seq = seq + [token]  
                candidate_score = score - math.log(prob)
                all_candidates.append((candidate_seq, candidate_score))
        if len(all_candidates) == 0:
            break  # No more extensions, stop generating

        # Sort all candidates by score and select top beam_width
        ordered = sorted(all_candidates, key=lambda tup: tup[1])
        sequences = ordered[:beam_width]
        
        # Stop if all sequences have no extensions
        if not sequences:
            break
    
    # Select the sequence with the lowest score
    if len(sequences) == 0:
        return []
    else:
        best_sequence,_ = min(sequences, key=lambda tup: tup[1])
        return best_sequence

In [ ]:
generated_tokens = beam_search_sampling(ngram_freq, n=2, beam_width=5, max_length=20)

print("\nGenerated Text:")
print(''.join(generated_tokens))

In [ ]:
n = 10
ngram4_freq = build_ngram_freq(bpe_tokens, n=n)
generated_tokens = beam_search_sampling(ngram4_freq, n=n, beam_width=5, max_length=20)

print("\nGenerated Text:")
print(''.join(generated_tokens))

In [ ]:
generated_tokens = beam_search_sampling(ngram4_freq, n=n, beam_width=5, max_length=100)
print("\nGenerated Text:")
print(''.join(generated_tokens))